In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/local/lib/python3.5/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
data = np.load("train.npy")

FileNotFoundError: [Errno 2] No such file or directory: 'train.npy'

In [ ]:
X = []
Y = []
i = 0
for elem in data:
    if i % 1000 == 0:
        print(i)
    X.append(tf.resize(elem[0], output_shape=(40,40)))
    Y.append(elem[1])
    i += 1

In [12]:
np.save('X', X)

In [13]:
np.save('Y', Y)

In [29]:
X = np.array(X)
Y = np.array(Y)

In [2]:
import numpy as np
import pandas as pd

X = np.load("X.npy")
Y = np.load("Y.npy")

X.resize((166708,40,40,1))

labels = np.unique(Y)
labels = labels.tolist()

for i in range(len(Y)):
    Y[i] = labels.index(Y[i])

X = X.astype(np.float16)
Y = Y.astype(np.int16)

## Data augmentation


In [31]:
X = np.load("X.npy")
Y = np.load("Y.npy")

In [7]:
from skimage.filters import threshold_adaptive
from skimage.morphology import skeletonize
from skimage.restoration import denoise_bilateral
plt.imshow(denoise_bilateral(X[0], multichannel=False), 'gray')
plt.figure()
plt.imshow(X[0], 'gray')

NameError: name 'X' is not defined

In [10]:
from tqdm import tqdm
from skimage.transform import rotate
X_denoise_test = []

for elem in tqdm(test_set):
    #X_denoise_test.append(rotate(denoise_bilateral(elem,multichannel=False),
                     #np.random.randint(low = -7, high=7) , resize=False))
    X_denoise_test.append(denoise_bilateral(elem,multichannel=False))

100%|██████████| 41428/41428 [01:12<00:00, 570.84it/s]  | 46/41428 [00:00<01:31, 453.04it/s]


In [62]:
np.save('X_denoise_rotate', X_denoise_rotate)

In [5]:
X_aug = []
Y_aug = []

for epoch in range(2):
    
    for i in range(len(X)):
        if i % 10000 == 0:
            print(epoch, i)
        temp = X[i]
        temp = noise_filter(temp, disk(np.random.random()))

        X_aug.append(rotate(X[i], np.random.randint(low = -30, high=30) , resize=False))
        X_aug.append(rotate(temp, np.random.randint(low = -30, high=30) , resize=False))
        Y_aug.append(Y[i])
        Y_aug.append(Y[i])

0 0


/usr/local/lib/python3.5/dist-packages/skimage/util/dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


0 10000
0 20000
0 30000
0 40000
0 50000
0 60000
0 70000
0 80000
0 90000
0 100000
0 110000
0 120000
0 130000
0 140000
0 150000
0 160000
1 0
1 10000
1 20000
1 30000
1 40000
1 50000
1 60000
1 70000
1 80000
1 90000
1 100000
1 110000
1 120000
1 130000
1 140000
1 150000
1 160000


In [12]:
size = int(len(X_aug)*7/10)
np.save('X_aug', X_aug[:size])
np.save('Y_aug', Y_aug)

In [2]:
X_denoise_rotate = np.load("X_denoise_rotate.npy")

In [13]:
X_denoise_test = np.array(X_denoise_test)
X_denoise_test.resize((len(X_denoise_test),40,40,1))

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [24]:
from sklearn.model_selection import train_test_split

X_train_aug, X_test_aug, y_train_aug, y_test_aug = train_test_split(X_aug, Y_aug[:int(len(Y_aug) * 7 / 10)], test_size=0.3, random_state=42)

In [4]:
X_flat = []

for elem in X:
    X_flat.append(elem.flatten())
X_flat = np.array(X_flat)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_jobs=-1, verbose=1, max_depth=15, n_estimators=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 13.3min


In [53]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import keras

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import PReLU
from keras.callbacks import ModelCheckpoint

from keras.callbacks import TensorBoard

batch_size = 512
nb_classes = 500
nb_epoch = 10

# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

input_shape = (40,40,1)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

prelu = keras.layers.advanced_activations.PReLU(init='zero', weights=None)

model = Sequential()


model.add(Convolution2D(48, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))

model.add(PReLU())
model.add(MaxPooling2D(pool_size=pool_size))

model.add(BatchNormalization())

model.add(Convolution2D(64, kernel_size[0], kernel_size[1]))
model.add(PReLU())
model.add(MaxPooling2D(pool_size=pool_size))

model.add(BatchNormalization())

model.add(Convolution2D(128, kernel_size[0], kernel_size[1]))
model.add(PReLU())
model.add(MaxPooling2D(pool_size=pool_size))

model.add(BatchNormalization())


model.add(Flatten())


model.add(Dense(650))
model.add(PReLU())

model.add(BatchNormalization())

model.add(Dense(550))
model.add(PReLU())

model.add(BatchNormalization())

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

tensorbord = TensorBoard(log_dir='/home/axcel/logs', histogram_freq=0,
                                         write_graph=True, write_images=False)

model_saver = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.6f}.hdf5",
                                              monitor='val_acc', verbose=0,
                                              save_best_only=True, save_weights_only=False,
                                              mode='auto')



datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2)

datagen.fit(X_train)

model.fit_generator(datagen.flow(X_train, Y_train, batch_size=256),
                    samples_per_epoch=len(X_train), nb_epoch=1, verbose=2,
                    validation_data=(X_test, Y_test), callbacks=[tensorbord, model_saver])

#model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          #verbose=2, validation_data=(X_test, Y_test), callbacks=[tensorbord])
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_11 (Convolution2D) (None, 38, 38, 48)    480         convolution2d_input_20[0][0]     
____________________________________________________________________________________________________
prelu_10 (PReLU)                 (None, 38, 38, 48)    69312       convolution2d_11[0][0]           
____________________________________________________________________________________________________
maxpooling2d_7 (MaxPooling2D)    (None, 19, 19, 48)    0           prelu_10[0][0]                   
____________________________________________________________________________________________________
batchnormalization_9 (BatchNorma (None, 19, 19, 48)    96          maxpooling2d_7[0][0]             
___________________________________________________________________________________________

In [18]:
X_aug = np.array(X_aug[:size], copy=False)
X_aug.resize((len(X_aug),40,40,1))

labels = np.unique(Y_aug)
labels = labels.tolist()

for i in range(len(Y_aug)):
    Y_aug[i] = labels.index(Y_aug[i])

X_aug = X_aug.astype(np.float16)
Y_aug = np.array(Y_aug).astype(np.int16)

In [9]:
from keras.models import load_model


model = load_model("./Models Architecture/4convpoolbigdense.hdf5")

In [ ]:
from keras.utils import np_utils
from keras.callbacks import TensorBoard

batch_size = 400
nb_classes = 500
nb_epoch = 10

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

input_shape = (40,40,1)

Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

tensorbord = TensorBoard(log_dir='/home/axcel/logs', histogram_freq=0,
                                         write_graph=True, write_images=True)

model_saver = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.6f}.hdf5",
                                              monitor='val_acc', verbose=0,
                                              save_best_only=True, save_weights_only=False,
                                              mode='auto')


model.fit_generator(datagen.flow(X_train, Y_train, batch_size=400),
                    samples_per_epoch=len(X_train), nb_epoch=280, verbose=2,
                    validation_data=(X_test, Y_test), callbacks=[tensorbord, model_saver] )
#model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=2,
          #verbose=2, validation_data=(X_test, Y_test), callbacks=[tensorbord])
#print('Test score:', score[0])
#print('Test accuracy:', score[1])

Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
Epoch 1/280
80s - loss: 0.2146 - acc: 0.9329 - val_loss: 0.5315 - val_acc: 0.8654
Epoch 2/280
79s - loss: 0.2108 - acc: 0.9349 - val_loss: 0.3427 - val_acc: 0.9081
Epoch 3/280
79s - loss: 0.2045 - acc: 0.9365 - val_loss: 0.1726 - val_acc: 0.9517
Epoch 4/280


In [70]:
model.save("4 conv with maxpool and batchnorm 2 dense sigmoid on augmented 50 epoch")

In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [7]:
from keras.models import load_model

#model = load_model("./Models Architecture/mnist_cnn_architecture_first_sub")

In [22]:
test = np.load('test.npy')

In [23]:
import skimage.transform

test_set = []
i = 0
for elem in test:
    if i % 1000 == 0:
        print(i)
    test_set.append(skimage.transform.resize(elem, output_shape=(40,40)))
    i += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000


In [24]:
test_set = np.array(test_set)

In [25]:
test_set.resize((41428,40,40,1))

In [14]:
ans = model.predict_classes(test_set, verbose=0)

In [ ]:
from keras.models import load_model
model1 = load_model("./Models Architecture/3 conv with maxpool and batchnorm with 2 dense 20 epoch")
model2 = load_model("./Models Architecture/3 conv with maxpool and batchnorm with 2 dense 20 epoch + 10 augmented")
model3 = load_model("./Models Architecture/4 conv with maxpool and batchnorm")
model4 = load_model("./Models Architecture/4 conv with maxpool and batchnorm V2")
model5 = load_model("./Models Architecture/4 conv with maxpool and batchnorm V2 200 epoch")
model6 = load_model("./Models Architecture/mnist_cnn_architecture_first_sub")
model7 = load_model("./Models Architecture/3 conv with maxpool and batchnorm with 2 dense 20 epoch + 30 denoised and rotated")
model8 = load_model("./Models Architecture/3 conv with maxpool and batchnorm mode1 with 2 dense 50 epoch")
model9 = load_model("./Models Architecture/3 conv with maxpool and batchnorm 2 dense on augmented 50 epoch")
model10 = load_model("./Models Architecture/3 conv with maxpool and batchnorm 2 dense_small on augmented 50 epoch")
model11 = load_model("./Models Architecture/4 conv with maxpool and batchnorm 2 dense on augmented 40 epoch")
model12 = load_model("./Models Architecture/4 conv with maxpool and batchnorm 2 dense sigmoid on augmented 50 epoch")
model13 = load_model("./Models Architecture/conv5.hdf5")
model14 = load_model("./Models Architecture/4convpoolbigdense.hdf5")
model15 = load_model("./Models Architecture/4convbatchmediumdense.hdf5")
model16 = load_model("./Models Architecture/3convprelu.hdf5")

In [58]:
prediction1 = model1.predict_proba(test_set, verbose=2)
prediction2 = model2.predict_proba(test_set, verbose=2)
prediction3 = model3.predict_proba(test_set, verbose=2)
prediction4 = model4.predict_proba(test_set, verbose=2)
prediction5 = model5.predict_proba(test_set, verbose=2)
prediction6 = model6.predict_proba(test_set, verbose=2)
prediction7 = model7.predict_proba(test_set, verbose=2)
prediction8 = model8.predict_proba(test_set, verbose=2)
prediction9 = model9.predict_proba(test_set, verbose=2)
prediction10 = model10.predict_proba(test_set, verbose=2)
prediction11 = model11.predict_proba(test_set, verbose=2)
prediction12 = model12.predict_proba(test_set, verbose=2)
prediction13 = model13.predict_proba(test_set, verbose=2)
prediction14 = model14.predict_proba(test_set, verbose=2)
prediction15 = model15.predict_proba(test_set, verbose=2)
prediction16 = model16.predict_proba(test_set, verbose=2)

In [ ]:
prediction1 = model1.predict_proba(X_test, verbose=2)
prediction2 = model2.predict_proba(X_test, verbose=2)
prediction3 = model3.predict_proba(X_test, verbose=2)
prediction4 = model4.predict_proba(X_test, verbose=2)
prediction5 = model5.predict_proba(X_test, verbose=2)
prediction6 = model6.predict_proba(X_test, verbose=2)
prediction7 = model7.predict_proba(X_test, verbose=2)
prediction8 = model8.predict_proba(X_test, verbose=2)
prediction9 = model9.predict_proba(X_test, verbose=2)
prediction10 = model10.predict_proba(X_test, verbose=2)
prediction11 = model11.predict_proba(X_test, verbose=2)
prediction12 = model12.predict_proba(X_test, verbose=2)
prediction13 = model13.predict_proba(X_test, verbose=2)
prediction14 = model14.predict_proba(X_test, verbose=2)
prediction15 = model15.predict_proba(X_test, verbose=2)
prediction16 = model16.predict_proba(X_test, verbose=2)

In [14]:
summury_predict = (prediction1 + prediction2 + prediction3 +
                   prediction4 + prediction5 + prediction6 +
                   prediction7 + prediction8 + prediction9 +
                   prediction10 + prediction11 + prediction12 +
                  prediction13 + prediction14 + prediction15) / 15

In [41]:
summury_predict = (best_alph[0]*prediction5 + best_alph[1]*prediction6 + best_alph[2]*prediction7
                 + best_alph[3]*prediction8 + best_alph[4]*prediction9 +
                   best_alph[5]*prediction10 + best_alph[6]*prediction11 + best_alph[7]*prediction12 +
                  best_alph[8]*prediction13 + best_alph[9]*prediction14)

In [59]:
summury_predict = (prediction5 +
                   prediction7 + prediction8 + prediction9 +
                   prediction10 + prediction11 + prediction12 +
                  prediction13 + prediction14 + prediction16)

In [ ]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score
alph_coef = [1, 0.7, 0.2]
best_alph = []
max_acc = 0


for i1 in tqdm(alph_coef):
    for i2 in alph_coef:
        for i3 in alph_coef:
            for i4 in alph_coef:
                for i5 in alph_coef:
                    for i6 in alph_coef:
                        for i7 in tqdm(alph_coef):
                            for i8 in alph_coef:
                                for i9 in alph_coef:
                                    for i10 in alph_coef:
                                        summury_predict = (i1*prediction5 + i2*prediction7
                                                           + i3*prediction8 + i4*prediction9 +
                                                           i5*prediction10 + i6*prediction11 +
                                                           i7*prediction12 + i8*prediction13 + i9*prediction14
                                                          + i10*prediction16)
                                        ans = []
                                        for elem in summury_predict:
                                            ans.append(np.argmax(elem))
                                        accuracy = accuracy_score(y_test, ans)
                                        if accuracy > max_acc:
                                            max_acc = accuracy
                                            best_alph = [i1,i2,i3,i4,i5,i6,i7,i8,i9,i10]

In [ ]:
max_acc

In [ ]:
best_alph

In [60]:
ans = []
for elem in summury_predict:
    ans.append(np.argmax(elem))

In [61]:
for i in range(len(ans)):
    ans[i] = labels[ans[i]]

In [62]:
ans = pd.DataFrame(ans)

In [63]:
temp = [i+1 for i in range(len(test_set))]

temp = pd.DataFrame(temp)

In [64]:
result = pd.concat([temp, ans], axis=1)

In [65]:
result.columns = ['Id', 'Category']

In [66]:
result = result.as_matrix()

In [67]:
# import numpy
numpy.savetxt("11netlatestprelu.csv", result, delimiter=",")